## kaggle 설치 및 폴더 추가

In [0]:
!pip install kaggle
!pip install hyperas
!pip install hyperopt
!mkdir .kaggle

## Kaggle Profile에서 Create New Api token 으로 json파일 받기

json 파일에서 username과 key 다운받기

In [0]:
import json
token = {"username":"juhk1017","key":"6c8f2081b6dc5d4f090e69dbd9436528"}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)

## 왠지 모르겠는데 이부분 2번 실행시켜줘야함

In [0]:
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json
!kaggle config set -n path -v{/content}

!chmod 600 /content/.kaggle/kaggle.json

## 원하는 dataset의 페이지에서 New Kernel 옆 ...을 클릭후 copy api command 로 코드 복사

In [0]:
!kaggle datasets download -d puneet6060/intel-image-classification -p /content #아마 kaggle datasets download -d puneet6060/intel-image-classification 이런식으로 복사 될거임

!unzip \intel-image-classification.zip
!unzip \seg_train.zip
!unzip \seg_test.zip
!unzip \seg_pred.zip

In [0]:
!ls #현재 디렉터리안에 Data file 들이 다운받아지고 압축이 풀려 있는 것을 확인할 수 있다.

## hyperas를 colab에서 사용하기에 필요

In [0]:
# Install the PyDrive wrapper & import libraries.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Copy/download the file
fid = drive.ListFile({'q':"title='Untitled3.ipynb'"}).GetList()[0]['id']
f = drive.CreateFile({'id': fid})
f.GetContentFile('Untitled3.ipynb')

In [0]:
import pandas as pd
import numpy as np
import os
from keras.models import Sequential,Model
from keras.layers import LSTM, Dense, TimeDistributed,CuDNNLSTM,BatchNormalization,Flatten,Dropout,Activation
from keras.layers import Input,concatenate

from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
import keras
from keras.layers.advanced_activations import LeakyReLU
from keras.regularizers import l2

from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plot
import tensorflow as tf
import cv2
import glob

from hyperas.distributions import uniform,choice
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim

In [0]:
def get_images(dir):
    images = []
    Labels = []
    
    for labels in os.listdir(dir):
        print(labels)
        if labels == 'buildings': 
            label = 0
        elif labels == 'forest':
            label = 1
        elif labels == 'glacier':
            label = 2
        elif labels == 'mountain':
            label = 3
        elif labels == 'sea':
            label = 4
        elif labels == 'street':
            label = 5
        #print(dir+ '/' + labels)
        for image_file in glob.glob(dir+ '/' + labels + '/*.jpg'): 
 
            image = cv2.imread(image_file)
            image = cv2.resize(image,(64,64)) 
        
            images.append(image)
            Labels.append(label)
        
        
    return shuffle(images,Labels)

In [0]:
images, Labels = get_images('seg_train')

images = np.array(images) #converting the list of images to numpy array.
Labels = np.array(Labels)

print("Shape of Images:",images.shape)
print("Shape of Labels:",Labels.shape)

In [0]:
    X_train, Y_train = get_images('seg_train')
    X_test, Y_test  = get_images('seg_test')
    Y_train = keras.utils.to_categorical(Y_train,num_classes=6, dtype='float32')
    Y_test = keras.utils.to_categorical(Y_test,num_classes=6, dtype='float32')

In [0]:
def data():

      X_train, Y_train = get_images('seg_train')
      X_test, Y_test  = get_images('seg_test')
      
      X_train = np.array(X_train)
      X_test = np.array(X_test)
      Y_train = np.array(Y_train)
      Y_test = np.array(Y_test)
      
      Y_train = keras.utils.to_categorical(Y_train,num_classes=6, dtype='float32')
      Y_test = keras.utils.to_categorical(Y_test,num_classes=6, dtype='float32')
      
      return X_train,Y_train,X_test,Y_test
  

In [0]:
def model(X_train,Y_train,X_test,Y_test):
    batch_size = 32
    nb_epoch = 20

    model = Sequential()
    
    model.add(Conv2D({{choice([128, 200, 256])}},kernel_size=(3,3),activation='relu',input_shape=(64,64,3)))
    model.add(Conv2D({{choice([128, 180, 256])}},kernel_size=(3,3),activation='relu'))
    model.add(MaxPooling2D(3,3))
    model.add(Conv2D({{choice([128, 180, 256])}},kernel_size=(3,3),activation='relu'))
    model.add(Conv2D({{choice([64, 140, 200])}},kernel_size=(3,3),activation='relu'))
    model.add(Conv2D({{choice([64, 100, 128])}},kernel_size=(3,3),activation='relu'))
    model.add(Conv2D({{choice([50, 64, 128])}},kernel_size=(3,3),activation='relu'))
    model.add(MaxPooling2D(3,3))
    model.add(Flatten())
    model.add(Dense(180,activation='relu'))
    model.add(Dense(100,activation='relu'))
    model.add(Dense(50,activation='relu'))
    model.add(Dropout({{uniform(0.2, 0.8)}}))
    model.add(Dense(6,activation='softmax'))
    model.summary()

    # let's train the model using SGD + momentum (how original).
    model.compile(loss='categorical_crossentropy',
                  optimizer=keras.optimizers.Adam(lr=0.0001),
                  metrics=['accuracy'])

    # fit the model on the batches generated by datagen.flow()
    model.fit(X_train,Y_train,batch_size=128,epochs=nb_epoch, verbose=1,validation_data=(X_test,Y_test))
    score, acc = model.evaluate(X_test, Y_test, verbose=0)

    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

In [0]:
  X_train, Y_train, X_test, Y_test = data()

  best_run, best_model = optim.minimize(model=model,
                                        data=data,
                                        algo=tpe.suggest,
                                        max_evals=4,
                                        functions=[get_images],
                                        notebook_name='Untitled3', #colab용
                                        trials=Trials())

  print("Evalutation of best performing model:")
  print(best_model.evaluate(X_test, Y_test))

In [0]:
input_1 = Input(shape=(150, 150, 3))

x_1 = Conv2D(64, kernel_size=3,kernel_regularizer=l2(0.001))(input_1)
x_1 = BatchNormalization()(x_1)
x_1 = Activation('relu')(x_1)
x_1 = MaxPooling2D(pool_size=(5, 5), strides=(2, 2))(x_1)

x_1 = Conv2D(64, kernel_size=3, kernel_regularizer=l2(0.0001))(x_1)
x_1 = BatchNormalization()(x_1)
x_1 = Activation('relu')(x_1)
x_1 = Conv2D(64, kernel_size=3,kernel_regularizer=l2(0.0001))(x_1)
x_1 = BatchNormalization()(x_1)
x_1 = Activation('relu')(x_1)
x_1 = Conv2D(64, kernel_size=3,kernel_regularizer=l2(0.0001))(x_1)
x_1 = BatchNormalization()(x_1)
x_1 = Activation('relu')(x_1)
x_1 = MaxPooling2D(pool_size=(5, 5), strides=(2, 2))(x_1)



x_1 = Flatten()(x_1)
x_1 = Dense(units=180)(x_1)
x_1 = BatchNormalization()(x_1)
x_1 = Activation('relu')(x_1)
x_1 = Dense(units=128)(x_1)
x_1 = Dense(units=128)(x_1)
x_1 = Dropout(0.4)(x_1)
x_1 = Dense(units=6,activation= 'softmax')(x_1)

In [0]:
cb_checkpoint = ModelCheckpoint(filepath='model_3.hdf5',
                                verbose=1,save_best_only=True)

In [0]:
model = Model(inputs=input_1,outputs=x_1)

model.compile(loss='categorical_crossentropy',
                optimizer=keras.optimizers.Adam(lr=0.0001),
                metrics=['accuracy'])

model.summary()

trained = model.fit(images,Labels,epochs=35,batch_size=128,validation_split=0.30,callbacks=[cb_checkpoint])


In [0]:
plot.plot(trained.history['acc'])
plot.plot(trained.history['val_acc'])
plot.title('Model accuracy')
plot.ylabel('Accuracy')
plot.xlabel('Epoch')
plot.legend(['Train', 'Test'], loc='upper left')
plot.show()

plot.plot(trained.history['loss'])
plot.plot(trained.history['val_loss'])
plot.title('Model loss')
plot.ylabel('Loss')
plot.xlabel('Epoch')
plot.legend(['Train', 'Test'], loc='upper left')
plot.show()

## TEST

In [0]:
img_test, test_label = get_images('seg_test')

img_test = np.array(img_test) 
test_label = np.array(test_label)

test_label = keras.utils.to_categorical(test_label,num_classes=6, dtype='float32')

print("Shape of Images:",img_test.shape)
print("Shape of Labels:",test_label.shape)

In [0]:

model = keras.models.load_model('model_2.hdf5') 
model.summary()
loss, acc = model.evaluate(img_test, test_label)
print(loss)
print(acc)

In [0]:
!ls

##Download


In [0]:
from google.colab import files
files.download('model_1.hdf5')

In [0]:
from google.colab import drive
drive.mount('/gdrive')

In [0]:
!ls /gdrive/My\ Drive/Colab\ Notebooks/

In [0]:
! cp model_2.hdf5 /gdrive/My\ Drive/Colab\ Notebooks/

In [0]:
!pwd

In [0]:
! cp /gdrive/My\ Drive/Colab\ Notebooks/model.hdf5 ./

In [0]:
model = keras.models.load_model('model.hdf5') 
model.summary()
loss, acc = model.evaluate(img_test, test_label)
print(loss)
print(acc)